In [1]:
import pandas as pd

In [2]:
data = pd.read_csv(r"fake_or_real_news.csv", engine="python")

In [3]:
data.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [4]:
print("Number of texts:", data.shape[0])

Number of texts: 6335


In [5]:
if data["label"][0] != 0 or 1:
  data["label"] = data["label"].replace("FAKE", 0)
  data["label"] = data["label"].replace("REAL", 1)


C:\Users\alber\AppData\Local\Temp\ipykernel_15000\58312266.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data["label"] = data["label"].replace("REAL", 1)


In [6]:
for row in data.title:
  if not row.endswith(". "):
    row = row + ". "
  # print(row)

In [7]:
data.text = data.title + data.text

In [8]:
data.columns

Index(['Unnamed: 0', 'title', 'text', 'label'], dtype='object')

In [9]:
data = data.drop(labels="title", axis=1)

In [10]:
data.rename(columns={"Unnamed: 0":"id"}, inplace=True)
data.shape

(6335, 3)

In [11]:
data.columns

Index(['id', 'text', 'label'], dtype='object')

In [12]:
data.id.duplicated().unique()


array([False])

In [13]:
# separating the data according to polarity because the TCT script does not keep the Polarity column

data.rename(columns={"label":"Polarity"}, inplace=True)
# fake_test = data[data.Polarity == 0]
# true_test = data[data.Polarity == 1]

In [14]:
# fake_test

In [15]:
# true_test

In [16]:

data.to_csv('fake_news_preprocess.csv', index=False)


Getting Subjectivity and Polarity scores

In [19]:
from textblob import TextBlob
import nltk
# nltk.download('vader_lexicon')
# nltk.download('punkt')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [20]:
sent = SentimentIntensityAnalyzer()

data["Overall_Sentiment"] = data["text"].apply(lambda row:sent.polarity_scores(row)["compound"])
data["Positive_Sentiment"] = data["text"].apply(lambda row:sent.polarity_scores(row)["neg"])
data["Negative_Sentiment"] = data["text"].apply(lambda row:sent.polarity_scores(row)["pos"])
data["Neutral_Sentiment"] = data["text"].apply(lambda row:sent.polarity_scores(row)["neu"])

data["Subjectivity"] = data["text"].apply(lambda row:TextBlob(row).sentiment[1])

#Fake News dataset Normalization

In [61]:
true_data_pre = pd.read_table('true_test.tsv', delimiter = '\t')

fake_data_pre = pd.read_table('fake_test.tsv', delimiter = '\t')

In [62]:
true_df = true_data_pre.drop(labels=["text_key", "id"], axis=1)
fake_df = fake_data_pre.drop(labels=["text_key", "id"], axis=1)



In [63]:
true_df["Polarity"] = 1
fake_df["Polarity"] = 0

In [64]:
dfs = [fake_df, true_df]
true_and_fake = pd.concat(dfs, axis = 0)

In [65]:
y = true_and_fake[["Polarity"]]
y

,Polarity
0,0
1,0
2,0
3,0
4,0
...,...
6337,1
6338,1
6339,1
6340,1


In [66]:
true_and_fake = true_and_fake.drop("Polarity", axis=1)

In [67]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

cols = list(true_and_fake.columns)

true_and_fake[cols] = scaler.fit_transform(true_and_fake[cols])

c:\Users\alber\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\preprocessing\_data.py:480: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
c:\Users\alber\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\preprocessing\_data.py:481: RuntimeWarning: All-NaN slice encountered
  data_max = np.nanmax(X, axis=0)


In [68]:
dfs = [true_and_fake, y]
df_scaled = pd.concat(dfs, axis = 1)
df_scaled

,DESPC,DESSC,DESWC,DESPL,DESPLd,DESPLw,DESSL,DESSLd,DESWLsy,DESWLsyd,...,WORD_PROPERTY_WRDHYPn,WORD_PROPERTY_WRDHYPv,WORD_PROPERTY_WRDHYPnv,WORD_PROPERTY_AOA,WORD_PROPERTY_AOA_MAX,WORD_PROPERTY_CONCRETENESS,WORD_PROPERTY_PREVALENCE,WORD_PROPERTY_PREVALENCE_MIN,WORD_SET_INCIDENCE_C4_COMMON_WORDS,Polarity
0,0.066524,0.114973,0.163885,0.011949,0.008981,0.009704,0.100626,0.055083,0.108689,0.138153,...,0.297788,0.192568,0.447776,0.387497,0.864543,0.388921,0.703376,0.703376,0.594802,0
1,0.000000,0.000000,0.000000,0.004061,NaN,0.000000,0.000000,NaN,0.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0
2,0.017167,0.033422,0.056790,0.012730,0.011417,0.012110,0.117941,0.050079,0.111677,0.134749,...,0.268293,0.256614,0.408195,0.370717,0.668614,0.432614,0.661672,0.661672,0.445675,0
3,0.000000,0.000000,0.000000,0.004061,NaN,0.000000,0.000000,NaN,0.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0
4,0.027897,0.029412,0.052948,0.007591,0.006916,0.007650,0.124694,0.104946,0.139152,0.152819,...,0.295022,0.265957,0.475238,0.440411,0.724853,0.410302,0.667496,0.667496,0.441176,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6337,0.000000,0.000000,0.000000,0.004061,NaN,0.000000,0.000000,NaN,0.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,1
6338,0.111588,0.056150,0.140713,0.006780,0.005163,0.009884,0.179874,0.073824,0.132160,0.148080,...,0.278280,0.258865,0.470854,0.403559,0.822817,0.441066,0.705736,0.705736,0.531969,1
6339,0.000000,0.000000,0.000000,0.004061,NaN,0.000000,0.000000,NaN,0.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,1
6340,0.103004,0.070856,0.107576,0.009385,0.007927,0.008090,0.106855,0.049176,0.089082,0.117580,...,0.331088,0.228448,0.443200,0.354220,0.758717,0.458203,0.690942,0.690942,0.442029,1


In [69]:
df_scaled["Polarity"]

0       0
1       0
2       0
3       0
4       0
       ..
6337    1
6338    1
6339    1
6340    1
6341    1
Name: Polarity, Length: 12665, dtype: int64

In [70]:
df_scaled.columns

Index(['DESPC', 'DESSC', 'DESWC', 'DESPL', 'DESPLd', 'DESPLw', 'DESSL',
       'DESSLd', 'DESWLsy', 'DESWLsyd', 'DESWLlt', 'DESWLltd', 'LDTTRc',
       'LDTTRa', 'LDMTLD', 'LDHDD', 'SYNLE', 'SYNNP', 'SYNMEDpos', 'SYNMEDwrd',
       'SYNMEDlem', 'SYNSTRUTa', 'SYNSTRUTt', 'RDFRE', 'READFKGL',
       'TOKEN_ATTRIBUTE_RATIO_ALHPA', 'TOKEN_ATTRIBUTE_RATIO_DIGIT',
       'TOKEN_ATTRIBUTE_RATIO_PUNCT', 'TOKEN_ATTRIBUTE_RATIO_URL',
       'TOKEN_ATTRIBUTE_RATIO_EMAIL', 'WORD_SET_INCIDENCE_WRDPRP1s',
       'WORD_SET_INCIDENCE_WRDPRP1p', 'WORD_SET_INCIDENCE_WRDPRP2',
       'WORD_SET_INCIDENCE_WRDPRP3s', 'WORD_SET_INCIDENCE_WRDPRP3p',
       'WORD_SET_INCIDENCE_CNCCaus', 'WORD_SET_INCIDENCE_CNCLogic',
       'WORD_SET_INCIDENCE_CNCTemp', 'WORD_SET_INCIDENCE_CNCAdd',
       'WORD_SET_INCIDENCE_CNCPos', 'WORD_SET_INCIDENCE_CNCNeg',
       'WORD_PROPERTY_WRDNOUN', 'WORD_PROPERTY_WRDVERB',
       'WORD_PROPERTY_WRDADJ', 'WORD_PROPERTY_WRDADV', 'WORD_PROPERTY_WRDFRQc',
       'WORD_PROPERTY_WRDFRQa'

In [71]:
data.columns

Index(['id', 'text', 'Polarity', 'Overall_Sentiment', 'Positive_Sentiment',
       'Negative_Sentiment', 'Neutral_Sentiment', 'Subjectivity'],
      dtype='object')

In [72]:
data = data.drop("Polarity", axis=1)

In [73]:
df_scaled = df_scaled.reset_index(drop=True)
df_scaled = pd.concat([df_scaled, data], axis =1)

In [74]:
df_scaled.columns

Index(['DESPC', 'DESSC', 'DESWC', 'DESPL', 'DESPLd', 'DESPLw', 'DESSL',
       'DESSLd', 'DESWLsy', 'DESWLsyd', 'DESWLlt', 'DESWLltd', 'LDTTRc',
       'LDTTRa', 'LDMTLD', 'LDHDD', 'SYNLE', 'SYNNP', 'SYNMEDpos', 'SYNMEDwrd',
       'SYNMEDlem', 'SYNSTRUTa', 'SYNSTRUTt', 'RDFRE', 'READFKGL',
       'TOKEN_ATTRIBUTE_RATIO_ALHPA', 'TOKEN_ATTRIBUTE_RATIO_DIGIT',
       'TOKEN_ATTRIBUTE_RATIO_PUNCT', 'TOKEN_ATTRIBUTE_RATIO_URL',
       'TOKEN_ATTRIBUTE_RATIO_EMAIL', 'WORD_SET_INCIDENCE_WRDPRP1s',
       'WORD_SET_INCIDENCE_WRDPRP1p', 'WORD_SET_INCIDENCE_WRDPRP2',
       'WORD_SET_INCIDENCE_WRDPRP3s', 'WORD_SET_INCIDENCE_WRDPRP3p',
       'WORD_SET_INCIDENCE_CNCCaus', 'WORD_SET_INCIDENCE_CNCLogic',
       'WORD_SET_INCIDENCE_CNCTemp', 'WORD_SET_INCIDENCE_CNCAdd',
       'WORD_SET_INCIDENCE_CNCPos', 'WORD_SET_INCIDENCE_CNCNeg',
       'WORD_PROPERTY_WRDNOUN', 'WORD_PROPERTY_WRDVERB',
       'WORD_PROPERTY_WRDADJ', 'WORD_PROPERTY_WRDADV', 'WORD_PROPERTY_WRDFRQc',
       'WORD_PROPERTY_WRDFRQa'

In [40]:
# df_scaled = df_scaled.T.drop_duplicates().T #removing the duplicate Polarity column
# del df_scaled["text"]

In [75]:
df_scaled.to_csv('test_data.csv')